In [2]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime

In [ ]:
!which chromedriver

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103


ConnectionError: HTTPConnectionPool(host='chromedriver.storage.googleapis.com', port=80): Max retries exceeded with url: /LATEST_RELEASE_83.0.4103 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x113d03a58>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [ ]:
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []

reviewURL_list = []


today = datetime.datetime.now()

for product in products:

    link = product.find('a')
    href = link['href']
    product_url = base_url + href
    productURL_list.append(product_url)
#     print(product_url)

    img = product.find('img')['src']
    productImage_list.append(img)
#     print(img)

    product_title = product.find('img')['alt']
    productTitle_list.append(product_title)
#     print(product_title)

    stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
    starReview_list.append(stars_review)
#     print(stars_review)
    
    review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
    reviewAmount_list.append(review_amount)
#     print(f"{review_amount} reviews")
    
    if int(review_amount) > 0:
        review_url = product.find('div', class_='stars').find('a')['href']
#         print(base_url + review_url)
    else:
        review_url = "NaN"
    reviewURL_list.append(review_url)

    price = product.find('span', class_='price-main')
    current_price = price.find('span', class_='visuallyhidden').text
    currentPrice_list.append(current_price)
#     print(current_price)
    
browser.quit()
    
data = {
    "Title": productTitle_list,
    "URL": productURL_list,
    "Image": productImage_list,
    "NumberOfStars": starReview_list,
    "NumberofReviews": reviewAmount_list,
    "ReviewsURL": reviewURL_list,
    "Price": currentPrice_list
       }

product_df = pd.DataFrame.from_dict(data)
product_df.head()

In [ ]:
browser.visit(product_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")